# Space Situational Awareness Demo


## Installing the Orbit Prediction Pipeline Tools

First we need to install the [orbit prediction package](https://github.com/IBM/spacetech-ssa/tree/master/orbit_prediction) from the [SSA project](https://github.com/ibm/spacetech-ssa) that will allow us to work with satellite orbit data.

In [ ]:
!pip install -e git+https://github.com/IBM/spacetech-ssa.git#egg=orbit_prediction\&subdirectory=orbit_prediction

## Getting TLE Data

[Two-line element set (TLE)](https://en.wikipedia.org/wiki/Two-line_element_set) is a data format that specifies the orbit of an object around the Earth at a particular point in time. The U.S. government provides an API for downloading TLE data and you can sign up for an account [here](https://www.space-track.org/auth/login). You should enter your account credentials in the cell below.

In [1]:
SPACETRACK_USERNAME='akshi.saxena8@gmail.com'
SPACETRACK_PASSWORD='Chinu!1234567890'

Next we will use the SpaceTrack API to download orbit data for the International Space Station (ISS) for the past 30 days. First we import the module that will allow us to ETL the SpaceTrack TLE data.

In [2]:
import orbit_prediction.spacetrack_etl as etl

Next we create a SpaceTrack API client. The SpaceTrack API is heavily rate limited and this client will keep us compliant with the terms of use.

In [3]:
spacetrack_client = etl.build_space_track_client(SPACETRACK_USERNAME,
                                                 SPACETRACK_PASSWORD)

We then create an instance that knows how to ETL the raw TLE data into a [pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [4]:
spacetrack_etl = etl.SpaceTrackETL(spacetrack_client)

Finally we fetch the raw TLE data and do the ETL. Every satellite in orbit has a [NORAD ID](https://en.wikipedia.org/wiki/Satellite_Catalog_Number) associated with it and the ID for the ISS is `25544`.

In [5]:
iss_orbit_data = spacetrack_etl.build_leo_df(norad_ids=['25544'],
                                             last_n_days=30,
                                             only_latest=None)

INFO:orbit_prediction.spacetrack_etl:Fetching Satellite Catalog Data...
INFO:orbit_prediction.spacetrack_etl:Number of TLE Batch Requests: 1
INFO:orbit_prediction.spacetrack_etl:Starting to fetch TLEs from space-track.org
INFO:orbit_prediction.spacetrack_etl:Processing batch 1/1
INFO:orbit_prediction.spacetrack_etl:Fetching TLEs for 1 ASOs...
INFO:orbit_prediction.spacetrack_etl:Parsing raw TLE data...
INFO:orbit_prediction.spacetrack_etl:Finished fetching TLEs
INFO:orbit_prediction.spacetrack_etl:Calculating orbital state vectors for 158 TLEs...


Lets take a look at the resulting DataFrame. Each line corresponds to a given observation of the ISS from the ground. Each observation consists of a timestamp the observation was made called the `epoch` and two 3-dimensional real valued vectors, **r** and **v**,

\begin{equation*}
  \mathbf{r} =
  \begin{pmatrix}
    r_{x}\\
    r_{y}\\
    r_{z}\\
  \end{pmatrix}
  \mathbf{v} =
  \begin{pmatrix}
    v_{x}\\
    v_{y}\\
    v_{z}\\
  \end{pmatrix}
\end{equation*}

corresponding to the position and velocity of the object respectively.

In [6]:
iss_orbit_data

aso_name aso_id                      epoch           r_x  \
0    ISS (ZARYA)  25544 2022-10-23 19:55:46.659360 -5.020808e+06   
1    ISS (ZARYA)  25544 2022-10-24 01:25:39.999936  4.801552e+06   
2    ISS (ZARYA)  25544 2022-10-24 04:54:22.459968 -1.494333e+06   
3    ISS (ZARYA)  25544 2022-10-24 09:27:34.233407  4.362309e+05   
4    ISS (ZARYA)  25544 2022-10-24 16:02:22.377984 -5.178598e+06   
..           ...    ...                        ...           ...   
153  ISS (ZARYA)  25544 2022-11-21 16:55:38.149823  4.058109e+06   
154  ISS (ZARYA)  25544 2022-11-21 21:32:05.444735  3.783627e+06   
155  ISS (ZARYA)  25544 2022-11-22 00:48:01.177344  4.144936e+06   
156  ISS (ZARYA)  25544 2022-11-22 05:41:55.807008  1.441518e+06   
157  ISS (ZARYA)  25544 2022-11-22 08:45:06.549696  2.150084e+06   

              r_y           r_z          v_x          v_y          v_z  \
0   -2.596349e+06  3.776583e+06   107.560056 -6377.113098 -4235.340887   
1    4.185331e+06 -2.360625e+06 -1828.788572  5132.339142  5386.809602   
2    4.629764e+06  4.741895e+06 -5498.770000 -4576.786976  2740.122624   
3    5.765796e+06  3.564440e+06 -5777.190542 -2321.156379  4465.960630   
4   -1.997819e+06  3.924585e+06  -588.700868 -6463.821407 -4060.762380   
..            ...           ...          ...          ...          ...   
153 -2.706654e+06  4.724260e+06  1437.425858  6999.191698  2770.715395   
154 -3.601986e+06  4.339214e+06  2164.584552  6474.565704  3481.166290   
155  8.598163e+05  5.308520e+06  -982.658298  7587.385196  -462.304910   
156  6.219977e+06  2.315209e+06 -4483.408106  3057.427457 -5409.769320   
157  5.653880e+06  3.086802e+06 -4100.779276  4236.439047 -4895.062326   

    object_type  
0       payload  
1       payload  
2       payload  
3       payload  
4       payload  
..          ...  
153     payload  
154     payload  
155     payload  
156     payload  
157     payload  

[158 rows x 10 columns]

A detailed description of every column is here:

| Field        | Description                                         | Type     |
|------------ |--------------------------------------------------- |-------- |
| aso\_id      | The unique ID for the ASO                           | string   |
| aso\_name    | The name of the ASO                                 | string   |
| epoch        | The timestamp the orbital observation was taken     | datetime |
| r\_x         | The \`x\` component of the position vector \`r\`    | float    |
| r\_y         | The \`y\` component of the position vector \`r\`    | float    |
| r\_z         | The \`z\` component of the position vector \`r\`    | float    |
| v\_x         | The \`x\` component of the velocity vector \`v\`    | float    |
| v\_y         | The \`y\` component of the velocity vector \`v\`    | float    |
| v\_z         | The \`z\` component of the velocity vector \`v\`    | float    |
| object\_type | Whether the ASO is a paylod, rocket body, or debris | string   |


## Building an ML Training Data Set

In this section we use the ISS orbit data from the last section to build a training data set for our machine learning models. We do this by using a physics-based orbital mechanics model to predict where it thinks the ISS will be for the observations that are 3 days in advance of a given row of our dataset. We can then compare where the ISS actually was based on the radar data and where the physics model says the ISS should have been. This value will be the error in the orbital mechanics model.

In [7]:
import orbit_prediction.build_training_data as training

This function uses a physics model to predict where the ISS will be based on the radar data.

In [8]:
physics_model_predicted_orbits = training.predict_orbits(iss_orbit_data,
                                                         last_n_days=None,
                                                         n_pred_days=3)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [01:58<00:00, 10.81s/it]


It added the following columns to our DataFrame:

| Field               | Description                                                                   | Type     |
|------------------- |----------------------------------------------------------------------------- |-------- |
| start\_epoch        | The \`epoch\` when the prediction was started                                 | datetime |
| start\_r\_x         | The \`x\` component of the position vector \`r\` where the prediction started | float    |
| start\_r\_y         | The \`y\` component of the position vector \`r\` where the prediction started | float    |
| start\_r\_z         | The \`z\` component of the position vector \`r\` where the prediction started | float    |
| start\_v\_x         | The \`x\` component of the velocity vector \`v\` where the prediction started | float    |
| start\_v\_y         | The \`y\` component of the velocity vector \`v\` where the prediction started | float    |
| start\_v\_z         | The \`z\` component of the velocity vector \`v\` where the prediction started | float    |
| elapsed\_seconds    | The number of seconds between the \`start\_epoch\` and \`epoch\`              | float    |
| physics\_pred\_r\_x | The \`x\` component of the predicted position vector \`r\`                    | float    |
| physics\_pred\_r\_y | The \`y\` component of the predicted position vector \`r\`                    | float    |
| physics\_pred\_r\_z | The \`z\` component of the predicted position vector \`r\`                    | float    |
| physics\_pred\_v\_x | The \`x\` component of the predicted velocity vector \`v\`                    | float    |
| physics\_pred\_v\_y | The \`y\` component of the predicted velocity vector \`v\`                    | float    |
| physics\_pred\_v\_z | The \`z\` component of the predicted velocity vector \`v\`                    | float    |

In the final part of this section, we calculate the error in the physical model.

In [9]:
physics_model_errors = training.calc_physics_error(physics_model_predicted_orbits)

Which adds the following to our training data:

| Field              | Description                                                        | Type  |
|------------------ |------------------------------------------------------------------ |----- |
| physics\_err\_r\_x | The prediction error in the \`x\` component of the position vector | float |
| physics\_err\_r\_y | The prediction error in the \`y\` component of the position vector | float |
| physics\_err\_r\_z | The prediction error in the \`z\` component of the position vector | float |
| physics\_err\_v\_x | The prediction error in the \`x\` component of the velocity vector | float |
| physics\_err\_v\_y | The prediction error in the \`y\` component of the velocity vector | float |
| physics\_err\_v\_z | The prediction error in the \`z\` component of the velocity vector | float |


## Training Gradient Boosted Regression Tree Models

Now that we have built a training dataset, our job is to build a machine learning model to predict each of the six `physics_err_` columns. Our baseline approach uses [gradient boosted](https://en.wikipedia.org/wiki/Gradient_boosting) [regression trees](https://en.wikipedia.org/wiki/Decision_tree_learning) (GBRTs) via the popular [XGBoost](https://xgboost.ai) package. First we split the training data we constructed in the previous section into a training and test set. We will use 80% of our data for training and reserve the remaining 20% to evaluate how well our model performs.

In [10]:
import orbit_prediction.ml_model as ml

train_test_data = ml.build_train_test_sets(physics_model_errors, test_size=0.2)

C:\Users\DEVANSH\anaconda3\envs\test\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Next we train six different regression models, one for each `physics_err_` column. Take a look [here](https://xgboost.readthedocs.io/en/latest/parameter.html) for all the possible parameters that can be passed to the underlying models.

In [11]:
gbrt_params = {'tree_method': 'hist'}
physics_error_model = ml.train_models(train_test_data, params=gbrt_params)

C:\Users\DEVANSH\anaconda3\envs\test\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Now that we have trained out models lets see how they performed using both the [root-mean-square error (RMSE)](https://en.wikipedia.org/wiki/Root-mean-square_deviation) and the [coefficient of determination (R<sup>2</sup>)](https://en.wikipedia.org/wiki/Coefficient_of_determination) as evaluation functions.

In [12]:
physics_error_model.eval_models(train_test_data['X_test'],
                                train_test_data['y_test'])

Error           RMSE       R^2
0  physics_err_r_x  305673.690457  0.696661
1  physics_err_r_y  259785.545782  0.694571
2  physics_err_r_z  282154.985782  0.574560
3  physics_err_v_x     471.522170  0.260930
4  physics_err_v_y     183.362447  0.874612
5  physics_err_v_z     185.859286  0.891688

Now can you improve upon these results by:

-   Using different parameters for the GBRT models?
-   Using a different machine learning technique?
-   Augmenting the training data or performing feature engineering?